In [1]:
import matplotlib.pyplot as plt
import numpy as np
import librosa
import wave
import pandas as pd
import soundfile as sf
import librosa.display
import IPython.display as ipd
from PIL import Image
import os
import scipy.signal as signal
from tqdm import tqdm
import random

In [2]:
# Step 1: Denoising using a low pass filter
def apply_low_pass_filter(audio, sampling_rate, cutoff_freq):
    nyquist_freq = 0.5 * sampling_rate
    normalized_cutoff_freq = cutoff_freq / nyquist_freq
    b, a = signal.butter(4, normalized_cutoff_freq, btype='low', analog=False)
    denoised_audio = signal.lfilter(b, a, audio)
    return denoised_audio

In [3]:
# Downsampling audio
def downsample_audio(audio,original_sampling_rate,target_sampling_rate):
    resampled_audio = librosa.resample(audio, orig_sr=original_sampling_rate, target_sr=target_sampling_rate)
    return resampled_audio

In [4]:
# Split audio into fixed-length segments
def split_audio(audio, segment_length):
    num_segments = len(audio) // segment_length
    segments = [audio[i*segment_length:(i+1)*segment_length] for i in range(num_segments)]
    return segments

In [5]:
def norm_mfcc(audio_path):

    # Load audio file
    audio, sr = librosa.load(audio_path, sr=None)

    # Compute MFCC features
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=48)

    # Normalize MFCC values
    normalized_mfcc = (mfcc - np.mean(mfcc)) / np.std(mfcc)
    return normalized_mfcc

In [6]:
def time_warp(mfcc, time_warping_range):
    num_frames = mfcc.shape[1]
    t = np.arange(num_frames)
    random_shift = np.random.randint(-time_warping_range, time_warping_range)
    warped_t = np.clip(t + random_shift, 0, num_frames - 1)
    warped_mfcc = mfcc[:, warped_t]
    return warped_mfcc[:, :48]

In [7]:
def frequency_mask(mfcc, frequency_masking_para, mask_max_frames):
    num_mfcc = mfcc.shape[0]
    num_frames = mfcc.shape[1]
    f = np.random.randint(0, frequency_masking_para)
    f0 = random.randint(0, num_mfcc - f)
    mfcc[f0:f0 + f, :] = 0.0
    return mfcc[:, :48]

In [8]:
input_shape = (39,32)  

In [9]:
MFCC_Dataset=[]

In [10]:
# Print the file names
def data(root_directory,final_class_labels):
    for folder_name, subfolders, filenames in os.walk(root_directory):
        # Iterate through audio files in the current folder
        for filename in filenames:
            if filename.lower().endswith(('.wav')):
#                 Construct the full path to the audio file
                audio_path = os.path.join(folder_name, filename)
                mfcc_array=norm_mfcc(audio_path)
                # Reshape MFCC array
                mfcc_reshaped = np.reshape(mfcc_array,input_shape)
                MFCC_Dataset.append([mfcc_reshaped,final_class_labels])

In [38]:
# Print the file names
def normal_data(root_directory,final_class_labels):
    for folder_name, subfolders, file_names in os.walk(root_directory):
        for file_name in file_names:
            if file_name.lower().endswith(('.wav')):
                audio_path = os.path.join(folder_name, file_name)
                mfcc_array=norm_mfcc(audio_path)
                # Reshape MFCC array
                print(mfcc_array.shape())
                mfcc_reshaped = np.reshape(mfcc_array, input_shape)
                MFCC_Dataset.append([mfcc_reshaped,final_class_labels])

                # Set time warping range (adjust according to your needs)
                time_warping_range = 5

                warped_mfcc = time_warp(mfcc_array, time_warping_range)
                # Reshape MFCC array
                mfcc_reshaped = np.reshape(warped_mfcc,input_shape)
                MFCC_Dataset.append([mfcc_reshaped,final_class_labels])

                # Set frequency masking parameters (adjust according to your needs)
                frequency_masking_para = 5
                mask_max_frames = 5

                masked_mfcc = frequency_mask(mfcc_array, frequency_masking_para, mask_max_frames)
                masked_mfcc2 = frequency_mask(mfcc_array, frequency_masking_para, mask_max_frames)
                # Reshape MFCC array
                mfcc_reshaped = np.reshape(masked_mfcc, input_shape)
                MFCC_Dataset.append([mfcc_reshaped,final_class_labels])
                mfcc_reshaped = np.reshape(masked_mfcc2, input_shape)
                MFCC_Dataset.append([mfcc_reshaped,final_class_labels])

In [12]:
data(r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\AugSegA\Artifact",'Artifact')
normal_data(r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\SplitA\Artifact",'Artifact')

In [13]:
data(r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\AugSegA\Extrahs",'Extrahs')
normal_data(r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\SplitA\Extrahs",'Extrahs')

In [40]:
data(r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\AugSegA\Murmur",'Murmur')
normal_data(r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\SplitA\Murmur",'Murmur')

In [41]:
data(r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\AugSegA\Normal",'Normal')
normal_data(r"C:\Users\Rashiqua Munshi\Desktop\HeartPS2\SplitA\Normal",'Normal')

In [16]:
len(MFCC_Dataset)

1222

In [17]:
MFCC_Df=pd.DataFrame(MFCC_Dataset, columns=['MFCC', 'Class'])

In [18]:
MFCC_Df

,MFCC,Class
0,"[[-3.1142056, -3.6126595, -6.192634, -6.212791...",artifact
1,"[[-6.322169, -6.40755, -6.569402, -6.644149, -...",artifact
2,"[[-4.500682, -4.984742, -6.77323, -6.7310724, ...",artifact
3,"[[-5.4014797, -5.7350993, -6.7023306, -6.67766...",artifact
4,"[[-4.9807153, -5.289154, -6.4026146, -6.405455...",artifact
...,...,...
1217,"[[-3.2540019, -3.8806489, -6.5898547, -6.68495...",normal
1218,"[[-5.724358, -6.1948705, -6.557403, -6.5594387...",normal
1219,"[[-5.724358, -5.724358, -5.724358, -5.724358, ...",normal
1220,"[[-5.724358, -6.1948705, -6.557403, -6.5594387...",normal


In [19]:
MFCC_Df['Class'].value_counts()

Class
artifact    720
extrahls    222
normal      148
murmur      132
Name: count, dtype: int64

In [20]:
for i in MFCC_Df["MFCC"]:
    print(i.shape)

(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(39, 32)
(

In [21]:
# Define the column value to filter
column_value = 'Artifact'

# Filter the DataFrame based on the column value
filtered_df = MFCC_Df[MFCC_Df['Class'] == column_value]

# Calculate the number of rows to drop (half of the filtered rows)
rows_to_drop = len(filtered_df) // 2

# Randomly select rows to drop
rows = np.random.choice(filtered_df.index, rows_to_drop, replace=False)

# Drop the selected rows
MFCC_Final = MFCC_Df.drop(rows)

# Print the updated DataFrame
print(MFCC_Final)

                                                   MFCC     Class
0     [[-3.1142056, -3.6126595, -6.192634, -6.212791...  artifact
1     [[-6.322169, -6.40755, -6.569402, -6.644149, -...  artifact
2     [[-4.500682, -4.984742, -6.77323, -6.7310724, ...  artifact
3     [[-5.4014797, -5.7350993, -6.7023306, -6.67766...  artifact
4     [[-4.9807153, -5.289154, -6.4026146, -6.405455...  artifact
...                                                 ...       ...
1217  [[-3.2540019, -3.8806489, -6.5898547, -6.68495...    normal
1218  [[-5.724358, -6.1948705, -6.557403, -6.5594387...    normal
1219  [[-5.724358, -5.724358, -5.724358, -5.724358, ...    normal
1220  [[-5.724358, -6.1948705, -6.557403, -6.5594387...    normal
1221  [[-5.724358, -6.1948705, -6.557403, -6.5594387...    normal

[1222 rows x 2 columns]


In [22]:
MFCC_Final['Class'].value_counts()

Class
artifact    720
extrahls    222
normal      148
murmur      132
Name: count, dtype: int64

In [23]:
X=np.array(MFCC_Final["MFCC"].tolist())
y=np.array(MFCC_Final["Class"].tolist())

In [24]:
X.shape

(1222, 39, 32)

In [25]:
input_shape = (39, 32, 3)
X = np.repeat(X[..., np.newaxis], 3, axis=-1)

In [26]:
np.unique(y)

array(['artifact', 'extrahls', 'murmur', 'normal'], dtype='<U8')

In [27]:
y=np.array(pd.get_dummies(y))

In [28]:
y

array([[ True, False, False, False],
       [ True, False, False, False],
       [ True, False, False, False],
       ...,
       [False, False, False,  True],
       [False, False, False,  True],
       [False, False, False,  True]])

In [29]:
X.shape

(1222, 39, 32, 3)

In [30]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [31]:
X_train.shape

(977, 39, 32, 3)

In [32]:
X_train,X_valid,y_train,y_valid=train_test_split(X_train,y_train,test_size=0.15,random_state=42)

In [33]:
X_train.shape

(830, 39, 32, 3)

In [34]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Conv2D,MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization

In [35]:
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout

# Load the pre-trained ResNet152V2 model
resnet152v2 = ResNet152V2(weights='imagenet', include_top=False, input_shape=(39, 32, 3))

# Freeze the pre-trained layers
for layer in resnet152v2.layers:
    layer.trainable = False

# Add custom layers on top of the pre-trained model
flatten_layer = Flatten()(resnet152v2.output)
dense_layer = Dense(128, activation='relu')(flatten_layer)
dropout_layer = Dropout(0.35)(dense_layer)
classification_layer = Dense(4, activation='softmax')(dropout_layer)  # Replace 'num_classes' with the number of classes in your problem

# Create the final model
model = Model(inputs=resnet152v2.input, outputs=classification_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Metal device set to: Apple M1
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 39, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 45, 38, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 20, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 22, 18, 64)   0           ['conv1_conv[0][0]']             
                                                                

                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 10, 8, 64)    16384       ['conv2_block3_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 10, 8, 64)   256         ['conv2_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 10, 8, 64)   0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_blo

 ization)                                                                                         
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, 5, 4, 128)   0           ['conv3_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, 5, 4, 512)    66048       ['conv3_block2_2_relu[0][0]']    
                                                                                                  
 conv3_block2_out (Add)         (None, 5, 4, 512)    0           ['conv3_block1_out[0][0]',       
                                                                  'conv3_block2_3_conv[0][0]']    
                                                                                                  
 conv3_blo

                                                                                                  
 conv3_block5_1_relu (Activatio  (None, 5, 4, 128)   0           ['conv3_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block5_2_pad (ZeroPaddin  (None, 7, 6, 128)   0           ['conv3_block5_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv3_block5_2_conv (Conv2D)   (None, 5, 4, 128)    147456      ['conv3_block5_2_pad[0][0]']     
                                                                                                  
 conv3_block5_2_bn (BatchNormal  (None, 5, 4, 128)   512         ['conv3_block5_2_conv[0][0]']    
 ization) 

 ormalization)                                                                                    
                                                                                                  
 conv3_block8_preact_relu (Acti  (None, 5, 4, 512)   0           ['conv3_block8_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv3_block8_1_conv (Conv2D)   (None, 5, 4, 128)    65536       ['conv3_block8_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv3_block8_1_bn (BatchNormal  (None, 5, 4, 128)   512         ['conv3_block8_1_conv[0][0]']    
 ization)                                                                                         
          

...

 lization)                                                                                        
                                                                                                  
 conv4_block29_1_relu (Activati  (None, 3, 2, 256)   0           ['conv4_block29_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block29_2_pad (ZeroPaddi  (None, 5, 4, 256)   0           ['conv4_block29_1_relu[0][0]']   
 ng2D)                                                                                            
                                                                                                  
 conv4_block29_2_conv (Conv2D)  (None, 3, 2, 256)    589824      ['conv4_block29_2_pad[0][0]']    
                                                                                                  
 conv4_blo

 conv4_block32_preact_bn (Batch  (None, 3, 2, 1024)  4096        ['conv4_block31_out[0][0]']      
 Normalization)                                                                                   
                                                                                                  
 conv4_block32_preact_relu (Act  (None, 3, 2, 1024)  0           ['conv4_block32_preact_bn[0][0]']
 ivation)                                                                                         
                                                                                                  
 conv4_block32_1_conv (Conv2D)  (None, 3, 2, 256)    262144      ['conv4_block32_preact_relu[0][0]
                                                                 ']                               
                                                                                                  
 conv4_block32_1_bn (BatchNorma  (None, 3, 2, 256)   1024        ['conv4_block32_1_conv[0][0]']   
 lization)

 lization)                                                                                        
                                                                                                  
 conv4_block34_2_relu (Activati  (None, 3, 2, 256)   0           ['conv4_block34_2_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block34_3_conv (Conv2D)  (None, 3, 2, 1024)   263168      ['conv4_block34_2_relu[0][0]']   
                                                                                                  
 conv4_block34_out (Add)        (None, 3, 2, 1024)   0           ['conv4_block33_out[0][0]',      
                                                                  'conv4_block34_3_conv[0][0]']   
                                                                                                  
 conv4_blo

 conv5_block1_1_bn (BatchNormal  (None, 2, 1, 512)   2048        ['conv5_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block1_1_relu (Activatio  (None, 2, 1, 512)   0           ['conv5_block1_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block1_2_pad (ZeroPaddin  (None, 4, 3, 512)   0           ['conv5_block1_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv5_block1_2_conv (Conv2D)   (None, 2, 1, 512)    2359296     ['conv5_block1_2_pad[0][0]']     
          

                                                                                                  
 conv5_block3_out (Add)         (None, 2, 1, 2048)   0           ['conv5_block2_out[0][0]',       
                                                                  'conv5_block3_3_conv[0][0]']    
                                                                                                  
 post_bn (BatchNormalization)   (None, 2, 1, 2048)   8192        ['conv5_block3_out[0][0]']       
                                                                                                  
 post_relu (Activation)         (None, 2, 1, 2048)   0           ['post_bn[0][0]']                
                                                                                                  
 flatten (Flatten)              (None, 4096)         0           ['post_relu[0][0]']              
                                                                                                  
 dense (De

In [36]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
checkpoint = ModelCheckpoint('heart_resnet_mfcc.hdf5', save_best_only=True, save_weights_only=False, monitor='val_accuracy', mode='max')
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_valid, y_valid), callbacks=[checkpoint])

Epoch 1/50


2023-07-14 12:58:24.704664: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


26/26 [==============================] - 22s 543ms/step - loss: 0.9924 - accuracy: 0.6265 - val_loss: 0.5707 - val_accuracy: 0.7891
Epoch 2/50
26/26 [==============================] - 5s 191ms/step - loss: 0.5551 - accuracy: 0.7855 - val_loss: 0.4504 - val_accuracy: 0.8367
Epoch 3/50
26/26 [==============================] - 6s 213ms/step - loss: 0.4496 - accuracy: 0.8253 - val_loss: 0.3772 - val_accuracy: 0.8639
Epoch 4/50
26/26 [==============================] - 5s 179ms/step - loss: 0.3999 - accuracy: 0.8482 - val_loss: 0.3424 - val_accuracy: 0.8912
Epoch 5/50
26/26 [==============================] - 5s 201ms/step - loss: 0.3808 - accuracy: 0.8301 - val_loss: 0.3147 - val_accuracy: 0.9116
Epoch 6/50
26/26 [==============================] - 3s 135ms/step - loss: 0.3155 - accuracy: 0.8855 - val_loss: 0.3093 - val_accuracy: 0.8980
Epoch 7/50
26/26 [==============================] - 4s 171ms/step - loss: 0.2871 - accuracy: 0.8952 - val_loss: 0.2811 - val_accuracy: 0.8912
Epoch 8/50
26/26

In [39]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.9469387531280518


In [62]:
def pred(audio_path, model_path):
    audio, sampling_rate = librosa.load(audio_path, sr=None)
    # Denoising
    cutoff_frequency = 195
    denoised_audio = apply_low_pass_filter(audio, sampling_rate, cutoff_frequency)

    # Downsampling
    target_sampling_rate = sampling_rate // 10
    downsampled_audio = downsample_audio(denoised_audio, sampling_rate, target_sampling_rate)

    # Splitting audio
    segment_length = target_sampling_rate * 3
    segments = split_audio(downsampled_audio, segment_length)
    
    model = keras.models.load_model(model_path)
    i=0
    
    for segment in segments:
        # Compute MFCC features
        mfcc = librosa.feature.mfcc(y=segment, sr=target_sampling_rate, n_mfcc=48)

        # Normalize MFCC values
        normalized_mfcc = (mfcc - np.mean(mfcc)) / np.std(mfcc)
        mfcc_reshaped = np.reshape(normalized_mfcc,(1,39,32))
        
        input_shape = (39, 32, 3)
        mfcc_final = np.repeat(mfcc_reshaped[..., np.newaxis], 3, axis=-1)
        
        prediction = model.predict(mfcc_final)
        x=np.argmax(prediction)
        confidence = prediction[0, x]
        i=i+1

#         prediction = model.predict(img_arr)
#         x=np.argmax(prediction)

        classes={0:'Artifact', 1:'Extrasystole', 2:'Murmur', 3:'Normal'}
        print(classes[x],confidence)
#         print(mfcc_final.shape)